# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from sa_models.SA_BERT_Dropout_Linear import SA_BERT_Dropout_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [7]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [8]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [9]:
BERT_FINE_TUNED_PATH = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [10]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_linear.csv'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
best_accuracy = 0.0

In [13]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [14]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [15]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [16]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = SA_BERT_Dropout_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.8918753862380981
Epoch: 0/2, Batch: 31/313, Loss: 0.389412522315979
Epoch: 0/2, Batch: 62/313, Loss: 0.44718050956726074
Epoch: 0/2, Batch: 93/313, Loss: 0.4466648995876312
Epoch: 0/2, Batch: 124/313, Loss: 0.1541673243045807
Epoch: 0/2, Batch: 155/313, Loss: 0.08768513053655624
Epoch: 0/2, Batch: 186/313, Loss: 0.06189978867769241
Epoch: 0/2, Batch: 217/313, Loss: 0.20630936324596405
Epoch: 0/2, Batch: 248/313, Loss: 0.44148629903793335
Epoch: 0/2, Batch: 279/313, Loss: 0.37325018644332886
Epoch: 0/2, Batch: 310/313, Loss: 0.09487621486186981
Epoch: 1/2, Batch: 0/313, Loss: 0.052659790962934494
Epoch: 1/2, Batch: 31/313, Loss: 0.10004114359617233
Epoch: 1/2, Batch: 62/313, Loss: 0.07608499377965927
Epoch: 1/2, Batch: 93/313, Loss: 0.05242089554667473
Epoch: 1/2, Batch: 124/313, Loss: 0.026612553745508194
Epoch: 1/2, Batch: 155/313, Loss: 0.03166494891047478
Epoch: 1/2, Batch: 186/313, Loss: 0.06774662435054779
Epoch: 1/2, Batch: 217/313, Loss

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 6/10
Epoch: 0/2, Batch: 0/313, Loss: 0.5273030996322632
Epoch: 0/2, Batch: 31/313, Loss: 0.2976996600627899
Epoch: 0/2, Batch: 62/313, Loss: 0.4517795443534851
Epoch: 0/2, Batch: 93/313, Loss: 0.17993727326393127
Epoch: 0/2, Batch: 124/313, Loss: 0.16028423607349396
Epoch: 0/2, Batch: 155/313, Loss: 0.6299176216125488
Epoch: 0/2, Batch: 186/313, Loss: 0.09512168169021606
Epoch: 0/2, Batch: 217/313, Loss: 0.08708690106868744
Epoch: 0/2, Batch: 248/313, Loss: 0.06157172843813896
Epoch: 0/2, Batch: 279/313, Loss: 0.2823922634124756
Epoch: 0/2, Batch: 310/313, Loss: 0.050207287073135376
Epoch: 1/2, Batch: 0/313, Loss: 0.04275844618678093
Epoch: 1/2, Batch: 31/313, Loss: 0.034653231501579285
Epoch: 1/2, Batch: 62/313, Loss: 0.37641841173171997
Epoch: 1/2, Batch: 93/313, Loss: 0.14036570489406586
Epoch: 1/2, Batch: 124/313, Loss: 0.053934261202812195
Epoch: 1/2, Batch: 155/313, Loss: 0.0366075299680233
Epoch: 1/2, Batch: 186/313, Loss: 0.04006796330213547
Epoch: 1/2, Batch: 217/313, Loss

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.5749421119689941
Epoch: 0/2, Batch: 31/313, Loss: 0.34619542956352234
Epoch: 0/2, Batch: 62/313, Loss: 0.2050846815109253
Epoch: 0/2, Batch: 93/313, Loss: 0.47545886039733887
Epoch: 0/2, Batch: 124/313, Loss: 0.0705590695142746
Epoch: 0/2, Batch: 155/313, Loss: 0.37134480476379395
Epoch: 0/2, Batch: 186/313, Loss: 0.34952616691589355
Epoch: 0/2, Batch: 217/313, Loss: 0.30032044649124146
Epoch: 0/2, Batch: 248/313, Loss: 0.06720657646656036
Epoch: 0/2, Batch: 279/313, Loss: 0.03426608443260193
Epoch: 0/2, Batch: 310/313, Loss: 0.06050623580813408
Epoch: 1/2, Batch: 0/313, Loss: 0.1882946789264679
Epoch: 1/2, Batch: 31/313, Loss: 0.05629577487707138
Epoch: 1/2, Batch: 62/313, Loss: 0.033900514245033264
Epoch: 1/2, Batch: 93/313, Loss: 0.03662695735692978
Epoch: 1/2, Batch: 124/313, Loss: 0.04080123454332352
Epoch: 1/2, Batch: 155/313, Loss: 0.02166314423084259
Epoch: 1/2, Batch: 186/313, Loss: 0.09928711503744125
Epoch: 1/2, Batch: 217/313, Loss

In [17]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.948718,0.948718,0.956388,0.948718,0.684895,0.948718,0.697263,200.450585
1,0.951923,0.951923,0.952768,0.951923,0.726105,0.951923,0.755830,195.854452
2,0.951923,0.951923,0.959196,0.951923,0.687908,0.951923,0.693350,194.716570
3,0.923077,0.923077,0.932640,0.923077,0.664640,0.923077,0.660842,194.679791
4,0.939103,0.939103,0.616116,0.939103,0.649236,0.939103,0.632201,194.734065
5,0.955128,0.955128,0.750440,0.955128,0.692844,0.955128,0.708917,194.740792
6,0.939103,0.939103,0.947947,0.939103,0.678760,0.939103,0.678013,194.728948
7,0.919872,0.919872,0.596154,0.919872,0.649388,0.919872,0.620402,194.549264
8,0.942308,0.942308,0.953641,0.942308,0.723351,0.942308,0.757983,194.336836
9,0.948718,0.948718,0.960974,0.948718,0.719728,0.948718,0.751210,194.689570


In [18]:
results.to_csv(STATS_OUTPUT)